In [22]:
!apt-get update
!apt-get install -y wget curl unzip
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt --fix-broken install -y

# Install ChromeDriver and automatically overwrite files
!wget https://chromedriver.storage.googleapis.com/114.0.5735.90/chromedriver_linux64.zip
!unzip -o chromedriver_linux64.zip  # -o flag to overwrite without asking
!mv chromedriver /usr/local/bin

# Install Selenium
!pip install selenium

Hit:1 https://dl.google.com/linux/chrome/deb stable InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 257 kB in 1s (175 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to

In [41]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

def get_tweet_page(tweet_id):
    url = f"https://twitter.com/username/status/{tweet_id}"

    # Set Chrome options for headless mode
    chrome_options = Options()
    chrome_options.add_argument('--headless')  # Headless mode, no browser UI
    chrome_options.add_argument('--no-sandbox')  # Solve certain security restrictions
    chrome_options.add_argument('--disable-dev-shm-usage')  # Disable shared memory
    chrome_options.add_argument('--disable-gpu')  # Disable GPU acceleration
    chrome_options.add_argument('--remote-debugging-port=9222')  # Enable debugging port

    # Use ChromeDriverManager to install and configure the driver automatically
    service = Service(ChromeDriverManager().install())

    # Launch Chrome browser with the specified options and service
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.get(url)

    # Wait for the page to load
    time.sleep(5)  # Wait 5 seconds for JavaScript rendering

    # Get the page source (HTML content)
    html = driver.page_source
    driver.quit()  # Close the browser
    return html


In [24]:
from bs4 import BeautifulSoup

def extract_tweet_data(html):
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Extract tweet text
    tweet_text = ''
    tweet_text_div = soup.find('div', {'data-testid': 'tweetText'})
    if tweet_text_div:
        tweet_text = tweet_text_div.get_text()

    # Extract image URLs
    images = []
    for img in soup.find_all('img'):
        img_url = img.get('src')
        if img_url and 'media' in img_url:  # Ensure it's an image URL
            images.append(img_url)

    return tweet_text, images


In [25]:
import requests

def download_images(images):
    for i, img_url in enumerate(images):
        try:
            img_data = requests.get(img_url).content
            with open(f"image_{i}.jpg", 'wb') as f:
                f.write(img_data)
            print(f"Downloaded image_{i}.jpg")
        except Exception as e:
            print(f"Error downloading image {i}: {e}")


In [48]:
def main(tweet_id):
    # Get the tweet's page HTML
    html = get_tweet_page(tweet_id)

    # Extract the tweet's text and image URLs
    tweet_text, images = extract_tweet_data(html)

    # Print the tweet text
    print(f"Tweet content: {tweet_text}")

    # Print and download the images if found
    if images:
        print("Images found:")
        for img in images:
            print(img)
        # Download the images
        download_images(images)
    else:
        print("No images found.")

if __name__ == '__main__':
    tweet_id = '1608960749485043715'  # Replace with the actual tweet ID
    main(tweet_id)


Tweet content: found #stablediffusion #dreamlikediffusion
Images found:
https://pbs.twimg.com/media/FlQtMCvXkAAgZBW?format=png&name=900x900
Downloaded image_0.jpg
